## Recipe Recommendation 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the dataset
data = pd.read_csv("recipes_ingredients.csv")

In [3]:
print(data.shape)
print(data.columns)
print(data.head(3))

(500471, 9)
Index(['id', 'name', 'description', 'ingredients', 'ingredients_raw', 'steps',
       'servings', 'serving_size', 'tags'],
      dtype='object')
       id                             name  \
0   71247          Cherry Streusel Cobbler   
1   76133  Reuben and Swiss Casserole Bake   
2  503816                 Yam-Pecan Recipe   

                                         description  \
0  I haven't made this in years, so I'm just gues...   
1  I think this is even better than a reuben sand...   
2  A lady I work with heard me taking about ZWT a...   

                                         ingredients  \
0  ["cherry pie filling", "condensed milk", "melt...   
1  ["corned beef chopped", "sauerkraut cold water...   
2  ["unsalted butter", "vegetable oil", "all - pu...   

                                     ingredients_raw  \
0  ["2 (21   ounce) cans   cherry pie filling","2...   
1  ["1/2-1   lb    corned beef, cooked and choppe...   
2  ["3/4  cup    unsalted butter, at roo

In [4]:
recipes = data[['id', 'name', 'ingredients', 'steps']]
print(recipes.head(3))

       id                             name  \
0   71247          Cherry Streusel Cobbler   
1   76133  Reuben and Swiss Casserole Bake   
2  503816                 Yam-Pecan Recipe   

                                         ingredients  \
0  ["cherry pie filling", "condensed milk", "melt...   
1  ["corned beef chopped", "sauerkraut cold water...   
2  ["unsalted butter", "vegetable oil", "all - pu...   

                                               steps  
0  ["Preheat oven to 375°F.", "Spread cherry pie ...  
1  ["Set oven to 350 degrees F.", "Butter a 9 x 1...  
2  ["Preheat oven to 350°F  In a mixing bowl, usi...  


In [5]:
print(recipes['ingredients'].iloc[0])

["cherry pie filling", "condensed milk", "melted margarine", "cinnamon", "nutmeg", "light brown sugar", "flour", "margarine", "chopped nuts", "oats", "butter - flavored cooking spray"]


### Convert ingredients_list into TF-IDF vectors.

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(recipes["ingredients"])

In [22]:
def recommend_recipes(user_ingredients, top_n):
    user_input = " ".join(user_ingredients)
    user_vec = vectorizer.transform([user_input])
    similarities = cosine_similarity(user_vec, X).flatten()

    top_indices = similarities.argsort()[-top_n:][::-1]
    results = recipes.iloc[top_indices][["name", "ingredients", "steps"]].copy()
    results["similarity"] = similarities[top_indices]

    for _, row in results.iterrows():
        print("="*60)
        print(f"🍽️  Recipe: {row['name']}")
        print(f"🔗 Similarity Score: {row['similarity']:.3f}\n")

        print("📝 Ingredients:")
        for ing in [row["ingredients"]]:
            print(f" - {str(ing).strip()}")

        print("\n👩‍🍳 Instructions:")
        for step in [row["steps"]]:
            print(f" - {str(step).strip()}")

        print("="*60, "\n")

    return results

In [23]:
user_ingredients = ['blueberries', 'yogurt', 'honey']
recommend_recipes(user_ingredients, top_n=5)

🍽️  Recipe: Blueberry Popsicles
🔗 Similarity Score: 0.725

📝 Ingredients:
 - ["fresh blueberries", "vanilla yogurt", "milk", "honey maple syrup"]

👩‍🍳 Instructions:
 - ["Blend blueberries, yogurt, milk, and sugar until smooth.", "Pour mixture into eight 5-ounce paper cups.", "Place foil over the top of each cup.", "Pierce the center of the foil and insert a wooden craft stick as the handle.", "Freeze up to six hours.", "Before serving, dip cups in warm water for a few seconds.", "Remove paper and foil.", "Send children outside to drip and make a mess. Stand at window and wave at them while you eat yours."]

🍽️  Recipe: Blueberry Milkshake
🔗 Similarity Score: 0.723

📝 Ingredients:
 - ["blueberries", "milk", "vanilla", "honey sugar"]

👩‍🍳 Instructions:
 - ["put all ingredients in a blender or a food processor and blend together until creamy.", "pour into cups and enjoy!"]

🍽️  Recipe: Honey Blueberry Spread
🔗 Similarity Score: 0.716

📝 Ingredients:
 - ["fresh blueberries frozen blueberri

,name,ingredients,steps,similarity
319651,Blueberry Popsicles,"[""fresh blueberries"", ""vanilla yogurt"", ""milk""...","[""Blend blueberries, yogurt, milk, and sugar u...",0.725340
68609,Blueberry Milkshake,"[""blueberries"", ""milk"", ""vanilla"", ""honey sugar""]","[""put all ingredients in a blender or a food p...",0.722718
484359,Honey Blueberry Spread,"[""fresh blueberries frozen blueberries"", ""hone...","[""Heat blueberries and 2 tablespoons honey in ...",0.716386
410442,Blueberry Cake Without Grains,"[""honey"", ""vanilla"", ""almonds chopped"", ""blueb...","[""Allow butter to soften or melt. Preheat ove...",0.674059
178190,Healthy Blueberry Milkshake,"[""bananas"", ""blueberries frozen blueberries"", ...","[""Place all ingredients into a blended."", ""Ble...",0.669613
